# Interactive haze removal for HDMI video stream
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://&lt;board_ip_address&gt;/lab for this notebook.
</div>

## Download the overlay

In [21]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

## Define Algorhms used by simplified DCP

In [22]:
import numpy as np

def get_radiance(image,atmosphere,transmition):
    radiance = np.true_divide(image-atmosphere,np.tile(transmition.reshape([image.shape[0],image.shape[1],1]),(1,1,3)))+atmosphere
    radiance[radiance>255] = 255
    radiance[radiance<0] = 0
    return radiance

def dehaze_fast(image,a=0.15):
    dark_channel = np.min(image,2)
    atmosphere = (1-a)*image[np.where(dark_channel == np.max(dark_channel))][0]
#     atmosphere = 225*np.array([1,1,1])
    transmition = 1 - dark_channel/255
    transmition[transmition<a] = a
    radiance = get_radiance(image,atmosphere,transmition)
    return np.uint8(radiance)

## Start HDMI streaming
please check your connection

In [23]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

In [24]:
hdmi_in.tie(hdmi_out)

# Process video
the input frame is resized to 180P to meet the real-time requirement

In [25]:
from ipywidgets import interact, FloatSlider
import threading
import time
import cv2

a = 0.15
a_slide = FloatSlider(min=0, max=1, step=0.01, value=0.15, description='a')
outframe = hdmi_out.newframe()

def adjust(a_slide):
    global a
    a = a_slide
    
class Thread_interact (threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        interact(adjust, a_slide=a_slide);
        while(1):
            time.sleep(0.1)
            
def play():
    inframe = hdmi_in.readframe()
    inframe_resize = cv2.resize(inframe,(320,180))
    result = dehaze_fast(inframe_resize, a)
    result = cv2.resize(result,(1920,1080))
    np.copyto(outframe,result)
    hdmi_out.writeframe(outframe)
            
class Thread_play (threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        while(1):
            play()
            time.sleep(0.01)

In [26]:
Thread_1 = Thread_interact()
Thread_2 = Thread_play()
Thread_1.start()
Thread_2.start()

interactive(children=(FloatSlider(value=0.15, description='a', max=1.0, step=0.01), Output()), _dom_classes=('…

<div class="alert alert-box alert-info">
Please directly use following code to stop threads, and restart from loading overlay
</div>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to stop the HDMI Video may hang the board 
when trying to download another bitstream onto the FPGA
</div>

In [27]:
hdmi_out.close()
hdmi_in.close()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-25-6c980857a2b0>", line 35, in run
    play()
  File "<ipython-input-25-6c980857a2b0>", line 28, in play
    hdmi_out.writeframe(outframe)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/video/hierarchies.py", line 391, in writeframe
    self._vdma.writechannel.writeframe(frame)
  File "/usr/local/lib/python3.6/dist-packages/pynq/lib/video/dma.py", line 454, in writeframe
    raise RuntimeError('DMA channel not started')
RuntimeError: DMA channel not started

